In [1]:
import os
import re
import string
import boto3
from botocore.exceptions import ClientError
import awswrangler as wr
import pandas as pd
import numpy as np

SEED = 1234
N_SAMPLES = 100_000
np.random.seed(SEED)

import gensim.downloader as api
from gensim import corpora
from gensim.utils import simple_preprocess
from gensim import models
from gensim.corpora import Dictionary
from gensim.matutils import corpus2dense, corpus2csc

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score
from sklearn.model_selection import GridSearchCV

import xgboost as xgb
from xgboost import XGBClassifier
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

from tqdm import tqdm

import warnings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 500)
pd.options.mode.chained_assignment = None
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
chunks = pd.read_csv('../data/toxic_data.csv', chunksize=100000)
df = pd.concat(chunks)
df.head()

,id,comment_text,split,created_date,publication_id,parent_id,article_id,rating,funny,wow,sad,likes,disagree,toxicity,severe_toxicity,obscene,sexual_explicit,identity_attack,insult,threat,male,female,transgender,other_gender,heterosexual,homosexual_gay_or_lesbian,bisexual,other_sexual_orientation,christian,jewish,muslim,hindu,buddhist,atheist,other_religion,black,white,asian,latino,other_race_or_ethnicity,physical_disability,intellectual_or_learning_disability,psychiatric_or_mental_illness,other_disability,identity_annotator_count,toxicity_annotator_count
0,1083994,He got his money... now he lies in wait till after the election in 2 yrs.... dirty politicians need to be afraid of Tar and feathers again... but they aren't and so the people get screwed.,train,2017-03-06 15:21:53.675241+00,21,NaN,317120,approved,0,0,0,2,0,0.373134,0.044776,0.089552,0.014925,0.000000,0.343284,0.014925,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,67
1,650904,Mad dog will surely put the liberals in mental hospitals. Boorah,train,2016-12-02 16:44:21.329535+00,21,NaN,154086,approved,0,0,1,2,0,0.605263,0.013158,0.065789,0.013158,0.092105,0.565789,0.065789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
2,5902188,And Trump continues his lifelong cowardice by not making this announcement himself.\n\nWhat an awful human being .....,train,2017-09-05 19:05:32.341360+00,55,NaN,374342,approved,1,0,2,3,7,0.666667,0.015873,0.031746,0.000000,0.047619,0.666667,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,63
3,7084460,"""while arresting a man for resisting arrest"".\n\nIf you cop-suckers can't see a problem with this, then go suck the barrel of a Glock.",test,2016-11-01 16:53:33.561631+00,13,NaN,149218,approved,0,0,0,0,0,0.815789,0.065789,0.552632,0.592105,0.000000,0.684211,0.105263,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,76
4,5410943,Tucker and Paul are both total bad ass mofo's.,train,2017-06-14 05:08:21.997315+00,21,NaN,344096,approved,0,0,0,1,0,0.550000,0.037500,0.337500,0.275000,0.037500,0.487500,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,80


In [3]:
df['comment_text'] = df['comment_text'].fillna("")

In [4]:
identity_columns = ['male', 'female', 'homosexual_gay_or_lesbian', 'christian', 'jewish', 'muslim', 'black', 'white', 'psychiatric_or_mental_illness']
for col in identity_columns + ['toxicity']:
    df.loc[:, col] = np.where(df[col] >= 0.5, True, False)

In [5]:
train_df = df[df['split'] == 'train']
test_df = df[df['split'] != 'train']

In [6]:
train_df.shape, test_df.shape

((1804875, 46), (194641, 46))

In [7]:
# sample = df
sample = train_df.sample(N_SAMPLES, random_state=SEED, ignore_index=True)
train_text, val_text, train_label, val_label = train_test_split(sample['comment_text'], sample['toxicity'], test_size=0.2, random_state=SEED)
# train_text, train_label = sample['comment_text'], sample['toxicity']
test_text, test_label = test_df['comment_text'], test_df['toxicity']

# train_label = train_label.astype('int').to_numpy()
# val_label = val_label.astype('int').to_numpy()
# test_label = test_label.astype('int').to_numpy()

In [8]:
val_df = df.iloc[val_text.index]
val_df.shape

(20000, 46)

In [9]:
train_text.shape, val_text.shape, test_text.shape

((80000,), (20000,), (194641,))

In [10]:
misspell_dict = {"aren't": "are not", "can't": "cannot", "couldn't": "could not",
                 "didn't": "did not", "doesn't": "does not", "don't": "do not",
                 "hadn't": "had not", "hasn't": "has not", "haven't": "have not",
                 "he'd": "he would", "he'll": "he will", "he's": "he is",
                 "i'd": "I had", "i'll": "I will", "i'm": "I am", "isn't": "is not",
                 "it's": "it is", "it'll": "it will", "i've": "I have", "let's": "let us",
                 "mightn't": "might not", "mustn't": "must not", "shan't": "shall not",
                 "she'd": "she would", "she'll": "she will", "she's": "she is",
                 "shouldn't": "should not", "that's": "that is", "there's": "there is",
                 "they'd": "they would", "they'll": "they will", "they're": "they are",
                 "they've": "they have", "we'd": "we would", "we're": "we are",
                 "weren't": "were not", "we've": "we have", "what'll": "what will",
                 "what're": "what are", "what's": "what is", "what've": "what have",
                 "where's": "where is", "who'd": "who would", "who'll": "who will",
                 "who're": "who are", "who's": "who is", "who've": "who have",
                 "won't": "will not", "wouldn't": "would not", "you'd": "you would",
                 "you'll": "you will", "you're": "you are", "you've": "you have",
                 "'re": " are", "wasn't": "was not", "we'll": " will", "tryin'": "trying"}


def _get_misspell(misspell_dict):
    misspell_re = re.compile('(%s)' % '|'.join(misspell_dict.keys()))
    return misspell_dict, misspell_re


def replace_typical_misspell(text):
    misspellings, misspellings_re = _get_misspell(misspell_dict)

    def replace(match):
        return misspellings[match.group(0)]

    return misspellings_re.sub(replace, text)
    

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']',
          '>', '%', '=', '#', '*', '+', '\\', '•', '~', '@', '£', '·', '_', '{', '}', '©', '^',
          '®', '`', '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 'Â', '█',
          '½', 'à', '…', '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶',
          '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼',
          '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲',
          'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪',
          '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√']


def clean_text(x):
    x = str(x)
    for punct in puncts + list(string.punctuation):
        if punct in x:
            x = x.replace(punct, f' {punct} ')
    return x


def clean_numbers(x):
    return re.sub('\d+', ' ', x)

In [11]:
# clean misspellings
train_text = train_text.apply(replace_typical_misspell)
val_text = val_text.apply(replace_typical_misspell)
test_text = test_text.apply(replace_typical_misspell)

# clean the text
train_text = train_text.apply(clean_text)
val_text = val_text.apply(replace_typical_misspell)
test_text = test_text.apply(clean_text)

# clean numbers
train_text = train_text.apply(clean_numbers)
val_text = val_text.apply(replace_typical_misspell)
test_text = test_text.apply(clean_numbers)

In [12]:
def simple_preproc(text):
    for line in text:
        yield simple_preprocess(line)

In [13]:
# train_lists = list(map(lambda x: simple_preprocess(x), train_text))
# test_lists = list(map(lambda x: simple_preprocess(x), test_text))  

In [14]:
# train_lists = [simple_preprocess(doc) for doc in tqdm(train_text)]
# # val_lists = [simple_preprocess(doc) for doc in val_text]
# test_lists = [simple_preprocess(doc) for doc in tqdm(test_text)]

In [15]:
# Create the Dictionary and Corpus
dictionary = corpora.Dictionary()

#allow_update=True - add new words to dictionary
bow_train = [dictionary.doc2bow(doc, allow_update=True) for doc in tqdm(simple_preproc(train_text))]
bow_val = [dictionary.doc2bow(doc, allow_update=False) for doc in tqdm(simple_preproc(val_text))]
bow_test = [dictionary.doc2bow(doc, allow_update=False) for doc in tqdm(simple_preproc(test_text))] 

dictionary.save("my_dictionary_full")
loaded_dict = corpora.Dictionary.load("my_dictionary_full")

80000it [00:09, 8410.48it/s]
20000it [00:02, 9883.55it/s] 
194641it [00:18, 10362.61it/s]


In [16]:
num_docs = dictionary.num_docs
num_terms = len(dictionary.keys())
print(f"Number of docs is {num_docs}, there are {num_terms} words in dictionary")

Number of docs is 80000, there are 66301 words in dictionary


In [17]:
tfidf = models.TfidfModel(bow_train, dictionary=loaded_dict)
train_tfidf = tfidf[bow_train]
val_tfidf = tfidf[bow_val]
test_tfidf = tfidf[bow_test]

In [18]:
train_tfidf_sparse = corpus2csc(train_tfidf, num_terms=num_terms, num_docs=num_docs).T
val_tfidf_sparse = corpus2csc(val_tfidf, num_terms=num_terms).T
test_tfidf_sparse = corpus2csc(test_tfidf, num_terms=num_terms).T

In [19]:
# Getting all memory using os.popen()
total_memory, used_memory, free_memory = map(
    int, os.popen('free -t -m').readlines()[-1].split()[1:])
  
# Memory usage
print("RAM memory % used:", round((used_memory/total_memory) * 100, 2))

RAM memory % used: 54.65


In [20]:
%%time
logreg = LogisticRegression(max_iter=1000)
logreg.fit(train_tfidf_sparse, train_label)
oof_name = 'predicted_target'
test_df[oof_name] = logreg.predict_proba(test_tfidf_sparse)[:, 1]

CPU times: user 11.1 s, sys: 13.4 s, total: 24.5 s
Wall time: 7.67 s


In [21]:
SUBGROUP_AUC = 'subgroup_auc'
BPSN_AUC = 'bpsn_auc'  # stands for background positive, subgroup negative
BNSP_AUC = 'bnsp_auc'  # stands for background negative, subgroup positive

def compute_auc(y_true, y_pred):
    try:
        return roc_auc_score(y_true, y_pred)
    except ValueError:
        return np.nan

def compute_subgroup_auc(df, subgroup, label, oof_name):
    subgroup_examples = df[df[subgroup]]
    return compute_auc(subgroup_examples[label], subgroup_examples[oof_name])

def compute_bpsn_auc(df, subgroup, label, oof_name):
    """Computes the AUC of the within-subgroup negative examples and the background positive examples."""
    subgroup_negative_examples = df[df[subgroup] & ~df[label]]
    non_subgroup_positive_examples = df[~df[subgroup] & df[label]]
    examples = subgroup_negative_examples.append(non_subgroup_positive_examples)
    return compute_auc(examples[label], examples[oof_name])

def compute_bnsp_auc(df, subgroup, label, oof_name):
    """Computes the AUC of the within-subgroup positive examples and the background negative examples."""
    subgroup_positive_examples = df[df[subgroup] & df[label]]
    non_subgroup_negative_examples = df[~df[subgroup] & ~df[label]]
    examples = subgroup_positive_examples.append(non_subgroup_negative_examples)
    return compute_auc(examples[label], examples[oof_name])

def compute_bias_metrics_for_model(dataset,
                                   subgroups,
                                   model,
                                   label_col,
                                   include_asegs=False):
    """Computes per-subgroup metrics for all subgroups and one model."""
    records = []
    for subgroup in subgroups:
        record = {
            'subgroup': subgroup,
            'subgroup_size': len(dataset[dataset[subgroup]])
        }
        record[SUBGROUP_AUC] = compute_subgroup_auc(dataset, subgroup, label_col, model)
        record[BPSN_AUC] = compute_bpsn_auc(dataset, subgroup, label_col, model)
        record[BNSP_AUC] = compute_bnsp_auc(dataset, subgroup, label_col, model)
        records.append(record)
    return pd.DataFrame(records).sort_values('subgroup_auc', ascending=True)
oof_name = 'predicted_target'
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, oof_name, 'toxicity')
bias_metrics_df

,subgroup,subgroup_size,subgroup_auc,bpsn_auc,bnsp_auc
2,homosexual_gay_or_lesbian,1065,0.741849,0.721643,0.938344
6,black,1519,0.757816,0.684025,0.955764
7,white,2452,0.787267,0.668299,0.968171
5,muslim,2040,0.790704,0.740633,0.945972
4,jewish,835,0.823446,0.833454,0.917689
8,psychiatric_or_mental_illness,511,0.854962,0.770391,0.957664
0,male,4386,0.857506,0.809902,0.948857
1,female,5155,0.863382,0.844982,0.935141
3,christian,4226,0.878163,0.903279,0.898506


In [22]:
def calculate_overall_auc(df, oof_name):
    true_labels = df['toxicity']
    predicted_labels = df[oof_name]
    return roc_auc_score(true_labels, predicted_labels)

def power_mean(series, p):
    total = sum(np.power(series, p))
    return np.power(total / len(series), 1 / p)

def get_final_metric(bias_df, overall_auc, POWER=-5, OVERALL_MODEL_WEIGHT=0.25):
    bias_score = np.average([
        power_mean(bias_df[SUBGROUP_AUC], POWER),
        power_mean(bias_df[BPSN_AUC], POWER),
        power_mean(bias_df[BNSP_AUC], POWER)
    ])
    return (OVERALL_MODEL_WEIGHT * overall_auc) + ((1 - OVERALL_MODEL_WEIGHT) * bias_score)
    
FINAL_SCORE = get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, oof_name))
print(f"FINAL SCORE IS {FINAL_SCORE}")

FINAL SCORE IS 0.8554141394637


In [23]:
import pickle
# # Save to file in the current working directory
pkl_filename = "logreg_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(logreg, file)

# Load from file
with open(pkl_filename, 'rb') as file:
    upload_pickle_model = pickle.load(file)


In [24]:
evaluation = [(train_tfidf_sparse, train_label), (val_tfidf_sparse, val_label)]

In [25]:
%%time
# fit model no training data
xgb_model = XGBClassifier(
    max_depth=9,
    eta=6.349718019312455e-05,
    gamma=0.000794047056078146,
    min_child_weight=3,
    subsample=0.8197963461187657,
    verbosity=1,
    colsample_bytree=0.6938357088977173,
    early_stopping_rounds=10, 
    objective='binary:logistic',
    )
xgb_model.fit(train_tfidf_sparse, train_label, eval_set=evaluation, verbose=False)

CPU times: user 4min 23s, sys: 493 ms, total: 4min 24s
Wall time: 2min 16s


XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.6938357088977173, early_stopping_rounds=10,
              enable_categorical=False, eta=6.349718019312455e-05,
              eval_metric=None, gamma=0.000794047056078146, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=6.34971802e-05,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=9,
              max_leaves=0, min_child_weight=3, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, ...)

In [26]:
xgb_model

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1,
              colsample_bytree=0.6938357088977173, early_stopping_rounds=10,
              enable_categorical=False, eta=6.349718019312455e-05,
              eval_metric=None, gamma=0.000794047056078146, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=6.34971802e-05,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=9,
              max_leaves=0, min_child_weight=3, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, ...)

In [27]:
xgb_model.save_model("model_xgb.json")

In [28]:
xgb_model2 = xgb.XGBClassifier()
xgb_model2.load_model("model_xgb.json")

In [29]:
oof_name = 'predicted_target_xgb'
test_df[oof_name] = xgb_model2.predict_proba(test_tfidf_sparse)[:, 1]

In [30]:
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, oof_name, 'toxicity')
bias_metrics_df

,subgroup,subgroup_size,subgroup_auc,bpsn_auc,bnsp_auc
2,homosexual_gay_or_lesbian,1065,0.577766,0.706692,0.585154
8,psychiatric_or_mental_illness,511,0.592464,0.710677,0.592200
5,muslim,2040,0.594053,0.704352,0.606074
6,black,1519,0.616558,0.613220,0.731965
3,christian,4226,0.645769,0.711515,0.645932
1,female,5155,0.649809,0.703959,0.659466
4,jewish,835,0.659815,0.699022,0.674633
7,white,2452,0.669070,0.432640,0.937753
0,male,4386,0.675718,0.682925,0.707190


In [31]:
FINAL_SCORE = get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, oof_name))
print(f"FINAL SCORE IS {FINAL_SCORE}")

FINAL SCORE IS 0.6460591629961917


In [32]:
# test_strings = ["you've fucked up Fd asshole $5", "55 go fuck yourself, fuckin' fuck. Die!@", "I am lesbian and I proud of it",
#                "BLM", "BLM sucks", "I think people like you should die", "I love you friends", "BLM $uck$",
#                "I hate software development", "I hate software development and you", 
#                "Go and kill this motherfucke"]

# for text in test_strings:
#     line = replace_typical_misspell(text)
#     line = clean_text(line)
#     line = clean_numbers(line)
#     bow = [dictionary.doc2bow(doc, allow_update=False) for doc in simple_preproc([line])]
#     test_tfidf = tfidf[bow]
#     test_tfidf_sparse = corpus2csc(test_tfidf, num_terms=num_terms).T
#     preds_lr = upload_pickle_model.predict_proba(test_tfidf_sparse)[:, 1]
#     preds_xgb = xgb_model2.predict_proba(test_tfidf_sparse)[:, 1]
#     print(f"Text: {text}, Logreg = {preds_lr[0]}, XGB = {preds_xgb[0]}")

In [33]:
import optuna

In [34]:
BEST_SCORE = 0.0

def objective(trial):
    dtrain = xgb.DMatrix(train_tfidf_sparse, train_label)
    dvalid = xgb.DMatrix(val_tfidf_sparse, val_label)
    
    global BEST_SCORE
    param = {
        "verbosity": 0,
        "objective": "binary:logistic",
        # use exact for small dataset.
        "tree_method": "exact",
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "gblinear", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1e-8, 1.0, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 1.0, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.2, 1.0),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 3, 9, step=2)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 2, 10)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    bst = xgb.train(param, dtrain)
    preds = bst.predict(dvalid)
    pred_labels = np.rint(preds)
    oof_name = 'predicted_target_xgb'
    val_df[oof_name] = pred_labels
    bias_metrics_df = compute_bias_metrics_for_model(val_df, identity_columns, oof_name, 'toxicity')
    final_score = get_final_metric(bias_metrics_df, calculate_overall_auc(val_df, oof_name))
    if final_score > BEST_SCORE:
        BEST_SCORE = final_score
        bst.save_model("xgb_optuna_best.json")
    
    return final_score

optuna.logging.set_verbosity(50)

In [35]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=25, timeout=600)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Number of finished trials:  25
Best trial:
  Value: 0.5034086640766879
  Params: 
    booster: dart
    lambda: 0.03339390366361633
    alpha: 0.0007534322027505882
    subsample: 0.3653595898664181
    colsample_bytree: 0.7360795085306511
    max_depth: 5
    min_child_weight: 5
    eta: 0.149201389860163
    gamma: 6.840880993257012e-06
    grow_policy: depthwise
    sample_type: weighted
    normalize_type: forest
    rate_drop: 0.00022776336562972275
    skip_drop: 0.040750331179617974
